In [4]:
import pandas as pd
import numpy as np

# Read in Data
data538 = pd.read_csv('nfl_elo.csv')
h = 65
my_dict = {}

def get_post_elo(home_team, home_team_elo, away_team, away_team_elo, point_diff, home_adv):
    
    if point_diff > 0:
        FD = 1-get_win_prob(home_team_elo, away_team_elo, home_adv)
    else: FD = 0 - get_win_prob(home_team_elo, away_team_elo, home_adv)
    
    home_post_elo = get_points(point_diff, home_team_elo, away_team_elo, home_adv)*FD  + home_team_elo
    away_post_elo = away_team_elo + (home_team_elo - home_post_elo)
    
    my_dict[home_team] = home_post_elo
    my_dict[away_team] = away_post_elo

    return home_post_elo, away_post_elo
    
# Probability of winning of home team
def get_win_prob(home_team_elo, away_team_elo, hfa):
    home_team_elo += hfa
    return 1.0/(1.0+(10.0**((away_team_elo-home_team_elo)/400)))

def get_points(point_diff, team_elo, opp_elo, HFA):
   
    if point_diff > 0:
        elo_dif = team_elo-opp_elo + HFA
    else: elo_dif = - (team_elo-opp_elo + HFA)

    return 20.0 * np.log(np.abs(point_diff)+1) * 2.2/(elo_dif*0.001+2.2)


games_2009 = data538[data538['season']==2009]
reg_season = games_2009[~np.isnan(games_2009.week)]

for i in range(reg_season.shape[0]):

    # Get Game Info
    row = reg_season.iloc[i]
    team = row.team1
    opp = row.team2
    point_diff = row.score1 - row.score2
    
    week = row.week
    
    if week == 1:
        team_pre = 1505
        opp_pre = 1505
    else:
        team_pre = my_dict[team]
        opp_pre = my_dict[opp]
      
    g = get_post_elo(team, team_pre, opp, opp_pre, point_diff, h)

print(my_dict)


{'PIT': 1542.4202637871315, 'TEN': 1518.549673664138, 'HOU': 1569.131343466702, 'NYJ': 1604.1764031354171, 'ARI': 1546.0444699164461, 'SF': 1521.805611406973, 'CLE': 1431.3495137646278, 'MIN': 1595.3276616653561, 'TB': 1393.1569752023252, 'DAL': 1636.561170170218, 'IND': 1604.1095707106003, 'JAX': 1420.8939033301667, 'NO': 1631.0483515649846, 'DET': 1285.0588068490495, 'ATL': 1541.2721717624377, 'MIA': 1459.0426814779494, 'NYG': 1471.3104631988572, 'WSH': 1398.9437091695745, 'BAL': 1567.8716495332089, 'KC': 1392.6465117670855, 'CIN': 1534.3936492443131, 'DEN': 1458.443470335721, 'GB': 1640.616801537635, 'CHI': 1467.5017844828824, 'SEA': 1377.9357024374187, 'LAR': 1290.095508703313, 'CAR': 1576.6704086533261, 'PHI': 1605.3577333306657, 'OAK': 1351.5231782309124, 'LAC': 1668.3026632602171, 'NE': 1591.2425337838963, 'BUF': 1467.19566045645}
